In [4]:
import requests

In [24]:
resposta = requests.post(url="https://gthc-phishing-url-bfcd29ec5f9c.herokuapp.com/api/v1/predict", json={"url" : "www.google.com"})
resposta.content

b'{"score":11}'